In [ ]:
import re
from selenium import webdriver
import time
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By


In [ ]:
file_path = "apartamentos_selenium.csv"
if os.path.exists(file_path):
    os.remove(file_path)


service = Service("C:\\Users\\luisp\\Desktop\\Drivers\\chromedriver-win64\\chromedriver.exe")

driver= webdriver.Chrome(service=service)

driver.get("https://mapainmueble.com")
time.sleep(2)

# URL base para cambiar de página automáticamente
base_url = "https://mapainmueble.com/apartamentos-en-venta/page/{}"

# Lista para almacenar datos de inmuebles
datos_inmuebles = []
max_paginas = 20  # Límite de páginas a scrapear

# Función para extraer datos de una propiedad
def extraer_detalles():
    try:
        collapse_section = driver.find_element(By.ID, "collapseOne")  # Buscar div con ID collapseOne
        detalles_seccion = collapse_section.find_element(By.CLASS_NAME, "panel-body")  # Encuentra el panel-body correcto
        detalles_items = detalles_seccion.find_elements(By.CSS_SELECTOR, "div.listing_detail")

        detalles_dict = {"URL": driver.current_url}  # Guardar URL de la propiedad

        for item in detalles_items:
            try:
                clave = item.find_element(By.TAG_NAME, "strong").text.strip().replace(":", "")
                valor = item.text.replace(clave, "").strip()
                detalles_dict[clave] = valor
                print(f"   ✅ {clave}: {valor}")  # Mostrar en tiempo real
            except:
                continue  # Si hay un error, saltar este dato

        return detalles_dict
    except:
        print("❌ No se encontró la sección de detalles.")
        return None

# 🔄 Loop para recorrer todas las páginas
for pagina in range(1, max_paginas + 1):
    url_pagina = base_url.format(pagina)
    print(f"\n📌 Cargando página {pagina}: {url_pagina}")
    driver.get(url_pagina)
    time.sleep(2)  # Espera rápida para cargar la página

    # 🔹 Encontrar todos los anuncios en la página actual
    listings = driver.find_elements(By.CSS_SELECTOR, "div.property_listing.property_card_default")
    links = []

    for listing in listings:
        try:
            link_tag = listing.find_element(By.CSS_SELECTOR, "h4 a")
            url = link_tag.get_attribute("href")
            links.append(url)
        except:
            continue

    if not links:
        print("🚫 No se encontraron más propiedades, terminando el scraping.")
        break  # Si no hay más propiedades, salimos del loop

    print(f"✅ Encontradas {len(links)} propiedades en esta página.")

    # 🔹 Ingresar a cada propiedad y extraer datos
    for idx, link in enumerate(links):
        driver.get(link)
        time.sleep(1.5)  # Espera corta para no perder datos

        print(f"\n🔍 Extrayendo datos de propiedad #{idx + 1}... ({link})")

        datos = extraer_detalles()
        if datos:
            datos_inmuebles.append(datos)

# Cerrar Selenium
driver.quit()

# Guardar datos en CSV
df = pd.DataFrame(datos_inmuebles)
df.to_csv("apartamentos_selenium.csv", index=False)
print("\n📁 Archivo 'inmuebles_completos.csv' generado con éxito.")

In [161]:
import pandas as pd
import os

file_path = "nuevo_apartamentos_selenium.csv"
if os.path.exists(file_path):
    os.remove(file_path)

In [162]:
df = pd.read_csv('apartamentos_selenium.csv')
df.head()

,URL,Precio,Tamaño de la propiedad,Parqueos,Dormitorios,Baños,ID de la propiedad,Tamaño del lote del Inmueble
0,https://mapainmueble.com/properties/apartament...,": $ 137,500 (GTQ 1,100,000.00)",: 75 m2,: 2,: 3,: 2,: DVA24953,NaN
1,https://mapainmueble.com/properties/amplio-apa...,": $ 550,000",: 254 m2,: 2,: 3,: 3,: CVA7490,: 10 m2
2,https://mapainmueble.com/properties/penthouse-...,": $ 1,000,000",: 400 m2,: 3,: 3,: 3.5,: DVA24775,NaN
3,https://mapainmueble.com/properties/apartament...,": $ 210,000 + Impuestos",: 179 m2,: 2,: 3,: 2,: DVA24766,NaN
4,https://mapainmueble.com/properties/venta-mode...,": $ 153,850 NEGOCIABLE",NaN,: 1,: 1,: 1,: DVA24758,NaN


In [163]:

df.drop(['URL','Tamaño del lote del Inmueble', 'ID de la propiedad'], axis=1, inplace=True)
df.head()

,Precio,Tamaño de la propiedad,Parqueos,Dormitorios,Baños
0,": $ 137,500 (GTQ 1,100,000.00)",: 75 m2,: 2,: 3,: 2
1,": $ 550,000",: 254 m2,: 2,: 3,: 3
2,": $ 1,000,000",: 400 m2,: 3,: 3,: 3.5
3,": $ 210,000 + Impuestos",: 179 m2,: 2,: 3,: 2
4,": $ 153,850 NEGOCIABLE",NaN,: 1,: 1,: 1


In [164]:
df['Precio'] = df['Precio'].astype(str)

# Filtra las filas para mantener solo formatos de precio válidos
df = df[df['Precio'].str.contains(r'^:\s*\$\s*\d+([\.,]\d+)?', regex=True)]

# Remueve el signo de dólar, los espacios y caracteres adicionales alrededor de los números
df['Precio'] = df['Precio'].str.replace(r'^:\s*\$\s*', '', regex=True)

# Muestra las primeras filas del dataframe resultante
df.head()

C:\Users\luisp\AppData\Local\Temp\ipykernel_26516\1252828661.py:4: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df = df[df['Precio'].str.contains(r'^:\s*\$\s*\d+([\.,]\d+)?', regex=True)]


,Precio,Tamaño de la propiedad,Parqueos,Dormitorios,Baños
0,"137,500 (GTQ 1,100,000.00)",: 75 m2,: 2,: 3,: 2
1,"550,000",: 254 m2,: 2,: 3,: 3
2,"1,000,000",: 400 m2,: 3,: 3,: 3.5
3,"210,000 + Impuestos",: 179 m2,: 2,: 3,: 2
4,"153,850 NEGOCIABLE",NaN,: 1,: 1,: 1


In [165]:
df = df[df['Precio'].str.contains(r'^\d+([\.,]\d+)?$', regex=True)]

df.head()

C:\Users\luisp\AppData\Local\Temp\ipykernel_26516\1613944272.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df = df[df['Precio'].str.contains(r'^\d+([\.,]\d+)?$', regex=True)]


,Precio,Tamaño de la propiedad,Parqueos,Dormitorios,Baños
1,"550,000",: 254 m2,: 2,: 3,: 3
15,"220,000",: 133 m2,: 1,: 2,: 2
19,"288,000",: 102 m2,: 2,: 2,: 2.5
20,"220,000",: 65 m2,: 1,: 2,: 2
21,"55,650",: 53 m2,: 1,: 3,: 1


In [166]:
# Asegurando que todas las columnas sean tratadas como strings para la limpieza y manejando valores no numéricos
df['Tamaño de la propiedad'] = pd.to_numeric(df['Tamaño de la propiedad'].astype(str).str.replace(': ', '').str.replace('m2', ''), errors='coerce')
df['Parqueos'] = pd.to_numeric(df['Parqueos'].astype(str).str.replace(': ', ''), errors='coerce')
df['Dormitorios'] = pd.to_numeric(df['Dormitorios'].astype(str).str.replace(': ', ''), errors='coerce').clip(lower=0)
df['Baños'] = pd.to_numeric(df['Baños'].astype(str).str.replace(': ', ''), errors='coerce').clip(lower=0)
df = df[df['Tamaño de la propiedad'] > 0]

# Llenar NaN con 0 o dejar como NaN dependiendo de tus necesidades
df.fillna(0, inplace=True)

# Muestra las primeras filas del dataframe resultante
df.head()



,Precio,Tamaño de la propiedad,Parqueos,Dormitorios,Baños
1,"550,000",254.0,2.0,3.0,3.0
15,"220,000",133.0,1.0,2.0,2.0
19,"288,000",102.0,2.0,2.0,2.5
20,"220,000",65.0,1.0,2.0,2.0
21,"55,650",53.0,1.0,3.0,1.0


In [167]:
df.dropna(subset=['Precio', 'Baños', 'Parqueos', 'Tamaño de la propiedad'], inplace=True)

df.head()

# Guardar el dataframe limpio
df.to_csv('nuevo_apartamentos_selenium.csv', index=False)